In [5]:
# Importing dependencies
import pandas as pd
import numpy as np

import re
import datetime
import string

import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.add_argument("--headless")

from sklearn.feature_extraction.text import CountVectorizer

import pickle

In [2]:
# # Testing current version of chrome driver
# driver = webdriver.Chrome()
# driver.get("https://www.google.com")

## 1 | Scrapping Data from The American Presidency Project
###### https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

sotu_raw_transcripts = []
presidents = []
sotu_dates = []

dates = list(range(1793, 2024)) # Dates 1793-2023
dates_str = [str(d) for d in dates]

for date in dates_str:
    print(f'Getting transcript for {date}')
    try:
        # Opening URL
        base_url = 'https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union'
        driver = webdriver.Chrome()
        driver.get(base_url)
        driver.implicitly_wait(10) # seconds
        element = driver.find_element(By.LINK_TEXT, date)
        webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
        current_url = driver.current_url # url containing transcript of SOTU addresses
        r = requests.get(current_url, headers=headers)
        print(r)
        # print(r.content)
        soup = BeautifulSoup(r.content, 'html.parser') 
        #print(soup.prettify())
        # Using BeautifulSoup to extract transcript
        # text = soup.find('div', class_='field-docs-content').get_text()
        text = soup.find('div', class_='field-docs-content').find_all('p')
        # Getting text content from <p> tags
        text = [t.get_text() for t in text]
        #text = soup.find('div', class_='node-documents').get_text()
        pres_name = soup.find('h3', class_='diet-title').get_text()
        date = soup.find('span', class_='date-display-single').get_text()
        sotu_raw_transcripts.append(text)
        presidents.append(pres_name)
        sotu_dates.append(date)
        driver.close()
    except:
        print(f'Could not get record for {date}')
#print(sotu_transcripts)
num_records = len(sotu_raw_transcripts)
print(f'Successfully scrapped {num_records} records')
driver.quit()

Getting transcript for 1793
<Response [200]>
Getting transcript for 1794
<Response [200]>
Getting transcript for 1795
<Response [200]>
Getting transcript for 1796
<Response [200]>
Getting transcript for 1797
<Response [200]>
Getting transcript for 1798
<Response [200]>
Getting transcript for 1799
<Response [200]>
Getting transcript for 1800
<Response [200]>
Getting transcript for 1801
<Response [200]>
Getting transcript for 1802
<Response [200]>
Getting transcript for 1803
<Response [200]>
Getting transcript for 1804
<Response [200]>
Getting transcript for 1805
<Response [200]>
Getting transcript for 1806
<Response [200]>
Getting transcript for 1807
<Response [200]>
Getting transcript for 1808
<Response [200]>
Getting transcript for 1809
<Response [200]>
Getting transcript for 1810
<Response [200]>
Getting transcript for 1811
<Response [200]>
Getting transcript for 1812
<Response [200]>
Getting transcript for 1813
<Response [200]>
Getting transcript for 1814
<Response [200]>
Getting tr

In [4]:
# Pickling raw transcripts
with open('./pickled_files/sotu_raw_transcripts.pkl', 'wb') as f:
    pickle.dump(sotu_raw_transcripts, f)

In [5]:
# Pickling list of presidents
with open('./pickled_files/presidents.pkl', 'wb') as f:
    pickle.dump(presidents, f)

In [6]:
# Pickling list of dates
with open('./pickled_files/dates.pkl', 'wb') as f:
    pickle.dump(sotu_dates, f)

In [7]:
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

# base_url = 'https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/annual-messages-congress-the-state-the-union'
# driver = webdriver.Chrome()
# driver.get(base_url)
# driver.implicitly_wait(10) # seconds
# element = driver.find_element_by_link_text('1793')
# webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
# current_url = driver.current_url # url containing transcript of SOTU addresses
# r = requests.get(current_url, headers=headers)
# print(r)
# # print(r.content)
# soup = BeautifulSoup(r.content, 'html.parser') 
# #print(soup.prettify())
# # Using BeautifulSoup to extract transcript
# text = soup.find('div', class_='field-docs-content').get_text()
# #text = soup.find('div', class_='node-documents').get_text()
# pres_name = soup.find('h3', class_='diet-title').get_text()
# date = soup.find('span', class_='date-display-single').get_text()

In [8]:
print('# of transcripts: ' + str(len(sotu_raw_transcripts)))
print('# of Presidents: ' + str(len(set(presidents))))
print('# of dates: ' + str(len(sotu_dates)))

# of transcripts: 229
# of Presidents: 43
# of dates: 229


## 2 | Data Augmentation

In [35]:
# Opening pickled list of raw transcripts
with open('./pickled_files/sotu_raw_transcripts.pkl', 'rb') as f:
    sotu_raw_transcripts = pickle.load(f)

In [36]:
# Opening pickled list of presidents
with open('./pickled_files/presidents.pkl', 'rb') as f:
    presidents = pickle.load(f)

In [37]:
# Opening pickled list of dates
with open('./pickled_files/dates.pkl', 'rb') as f:
    sotu_dates = pickle.load(f)

In [38]:
print(len(sotu_raw_transcripts))
print(len(presidents))
print(len(sotu_dates))

229
229
229


In [39]:
sotu_speeches = pd.DataFrame({'sotu_date': sotu_dates, 'president_name': presidents, 'raw_text': sotu_raw_transcripts})

In [40]:
sotu_speeches.head()

,sotu_date,president_name,raw_text
0,"December 03, 1793",George Washington,[Fellow-Citizens of the Senate and House of Re...
1,"November 19, 1794",George Washington,[Fellow-Citizens of the Senate and House of Re...
2,"December 08, 1795",George Washington,[Fellow-Citizens of the Senate and House of Re...
3,"December 07, 1796",George Washington,[Fellow-Citizens of the Senate and House of Re...
4,"November 22, 1797",John Adams,[Gentlemen of the Senate and Gentlemen of the ...


In [41]:
sotu_speeches.president_name.unique()

array(['George Washington', 'John Adams', 'Thomas Jefferson',
       'James Madison', 'James Monroe', 'John Quincy Adams',
       'Andrew Jackson', 'Martin van Buren', 'John Tyler',
       'James K. Polk', 'Zachary Taylor', 'Millard Fillmore',
       'Franklin Pierce', 'James Buchanan', 'Abraham Lincoln',
       'Andrew Johnson', 'Ulysses S. Grant', 'Rutherford B. Hayes',
       'Chester A. Arthur', 'Grover Cleveland', 'Benjamin Harrison',
       'William McKinley', 'Theodore Roosevelt', 'William Howard Taft',
       'Woodrow Wilson', 'Warren G. Harding', 'Calvin Coolidge',
       'Herbert Hoover', 'Franklin D. Roosevelt', 'Harry S. Truman',
       'Dwight D. Eisenhower', 'John F. Kennedy', 'Lyndon B. Johnson',
       'Richard Nixon', 'Gerald R. Ford', 'Jimmy Carter', 'Ronald Reagan',
       'George Bush', 'William J. Clinton', 'George W. Bush',
       'Barack Obama', 'Donald J. Trump', 'Joseph R. Biden'], dtype=object)

In [42]:
republican = ['Abraham Lincoln',
'Ulysses S. Grant',
'Rutherford B. Hayes',
'James Garfield',
'Chester A. Arthur',
'Benjamin Harrison',
'William McKinley',
'Theodore Roosevelt',
'William Howard Taft',
'Warren G. Harding',
'Calvin Coolidge',
'Herbert Hoover',
'Dwight D. Eisenhower',
'Richard M. Nixon',
'Gerald R. Ford',
'Ronald Reagan',
'George Bush',
'George W. Bush',
'Donald J. Trump']
republican_dict = { r : 'republican' for r in republican }
republican_dict

{'Abraham Lincoln': 'republican',
 'Ulysses S. Grant': 'republican',
 'Rutherford B. Hayes': 'republican',
 'James Garfield': 'republican',
 'Chester A. Arthur': 'republican',
 'Benjamin Harrison': 'republican',
 'William McKinley': 'republican',
 'Theodore Roosevelt': 'republican',
 'William Howard Taft': 'republican',
 'Warren G. Harding': 'republican',
 'Calvin Coolidge': 'republican',
 'Herbert Hoover': 'republican',
 'Dwight D. Eisenhower': 'republican',
 'Richard M. Nixon': 'republican',
 'Gerald R. Ford': 'republican',
 'Ronald Reagan': 'republican',
 'George Bush': 'republican',
 'George W. Bush': 'republican',
 'Donald J. Trump': 'republican'}

In [43]:
democrat = ['Andrew Jackson',
'Martin van Buren',
'James K. Polk',
'Franklin Pierce',
'James Buchanan',
'Grover Cleveland',
'Woodrow Wilson',
'Franklin D. Roosevelt',
'Harry S. Truman',
'John F. Kennedy',
'Lyndon B. Johnson',
'Jimmy Carter',
'William J. Clinton',
'Barack Obama',
'Joseph R. Biden']
democrat_dict = { d : 'democrat' for d in democrat }
democrat_dict

{'Andrew Jackson': 'democrat',
 'Martin van Buren': 'democrat',
 'James K. Polk': 'democrat',
 'Franklin Pierce': 'democrat',
 'James Buchanan': 'democrat',
 'Grover Cleveland': 'democrat',
 'Woodrow Wilson': 'democrat',
 'Franklin D. Roosevelt': 'democrat',
 'Harry S. Truman': 'democrat',
 'John F. Kennedy': 'democrat',
 'Lyndon B. Johnson': 'democrat',
 'Jimmy Carter': 'democrat',
 'William J. Clinton': 'democrat',
 'Barack Obama': 'democrat',
 'Joseph R. Biden': 'democrat'}

In [44]:
federalist = ['George Washington',
'John Adams']
federalist_dict = { f : 'federalist' for f in federalist }
federalist_dict

{'George Washington': 'federalist', 'John Adams': 'federalist'}

In [45]:
democratic_republican = ['Thomas Jefferson',
'James Madison',
'James Monroe',
'John Quincy Adams']
democratic_republican_dict = { dr : 'democratic_republican' for dr in democratic_republican }
democratic_republican_dict

{'Thomas Jefferson': 'democratic_republican',
 'James Madison': 'democratic_republican',
 'James Monroe': 'democratic_republican',
 'John Quincy Adams': 'democratic_republican'}

In [46]:
whig = ['William Henry Harrison',
'John Tyler',
'Zachary Taylor',
'Millard Fillmore']
whig_dict = { w : 'democratic_republican' for w in whig }
whig_dict

{'William Henry Harrison': 'democratic_republican',
 'John Tyler': 'democratic_republican',
 'Zachary Taylor': 'democratic_republican',
 'Millard Fillmore': 'democratic_republican'}

In [47]:
union = ['Andrew Johnson']
union_dict = {u : 'union' for u in union}
union_dict

{'Andrew Johnson': 'union'}

In [48]:
# Merging contents of all dictionaries
presidential_party_dict = {**republican_dict, **democrat_dict, **federalist_dict, **democratic_republican_dict, **whig_dict, **union_dict}    
presidential_party_dict

{'Abraham Lincoln': 'republican',
 'Ulysses S. Grant': 'republican',
 'Rutherford B. Hayes': 'republican',
 'James Garfield': 'republican',
 'Chester A. Arthur': 'republican',
 'Benjamin Harrison': 'republican',
 'William McKinley': 'republican',
 'Theodore Roosevelt': 'republican',
 'William Howard Taft': 'republican',
 'Warren G. Harding': 'republican',
 'Calvin Coolidge': 'republican',
 'Herbert Hoover': 'republican',
 'Dwight D. Eisenhower': 'republican',
 'Richard M. Nixon': 'republican',
 'Gerald R. Ford': 'republican',
 'Ronald Reagan': 'republican',
 'George Bush': 'republican',
 'George W. Bush': 'republican',
 'Donald J. Trump': 'republican',
 'Andrew Jackson': 'democrat',
 'Martin van Buren': 'democrat',
 'James K. Polk': 'democrat',
 'Franklin Pierce': 'democrat',
 'James Buchanan': 'democrat',
 'Grover Cleveland': 'democrat',
 'Woodrow Wilson': 'democrat',
 'Franklin D. Roosevelt': 'democrat',
 'Harry S. Truman': 'democrat',
 'John F. Kennedy': 'democrat',
 'Lyndon B. John

In [49]:
# Pickling party affiliations
with open('./pickled_files/presidential_party_dict.pkl', 'wb') as f:
    pickle.dump(presidential_party_dict, f)

In [50]:
sotu_speeches['president_party'] = sotu_speeches['president_name'].map(presidential_party_dict)


In [51]:
sotu_speeches.head()

,sotu_date,president_name,raw_text,president_party
0,"December 03, 1793",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
1,"November 19, 1794",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
2,"December 08, 1795",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
3,"December 07, 1796",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
4,"November 22, 1797",John Adams,[Gentlemen of the Senate and Gentlemen of the ...,federalist


In [52]:
# Pickling raw dataframe
with open('./pickled_files/raw_sotu_speeches.pkl', 'wb') as f:
    pickle.dump(sotu_speeches, f)

## 3 | Data Cleaning

In [6]:
# Opening pickled dataframe
with open('./pickled_files/raw_sotu_speeches.pkl', 'rb') as f:
    raw_sotu_speeches = pickle.load(f)

In [7]:
raw_sotu_speeches.head()

,sotu_date,president_name,raw_text,president_party
0,"December 03, 1793",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
1,"November 19, 1794",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
2,"December 08, 1795",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
3,"December 07, 1796",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
4,"November 22, 1797",John Adams,[Gentlemen of the Senate and Gentlemen of the ...,federalist


In [8]:
clean_sotu_speeches = raw_sotu_speeches.copy()
clean_sotu_speeches.head()

,sotu_date,president_name,raw_text,president_party
0,"December 03, 1793",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
1,"November 19, 1794",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
2,"December 08, 1795",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
3,"December 07, 1796",George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist
4,"November 22, 1797",John Adams,[Gentlemen of the Senate and Gentlemen of the ...,federalist


In [9]:
# Converting date to datetime object and extracting month from date
clean_sotu_speeches.sotu_date = clean_sotu_speeches.sotu_date.astype('datetime64[ns]')
clean_sotu_speeches['year'] = pd.DatetimeIndex(clean_sotu_speeches.sotu_date).year
clean_sotu_speeches.head()

,sotu_date,president_name,raw_text,president_party,year
0,1793-12-03,George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist,1793
1,1794-11-19,George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist,1794
2,1795-12-08,George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist,1795
3,1796-12-07,George Washington,[Fellow-Citizens of the Senate and House of Re...,federalist,1796
4,1797-11-22,John Adams,[Gentlemen of the Senate and Gentlemen of the ...,federalist,1797


In [10]:
# Merging the contents of the list of raw transcripts into a single string
clean_sotu_speeches.raw_text = clean_sotu_speeches.raw_text.apply(lambda x: ''.join(x))
clean_sotu_speeches.raw_text

0      Fellow-Citizens of the Senate and House of Rep...
1      Fellow-Citizens of the Senate and House of Rep...
2      Fellow-Citizens of the Senate and House of Rep...
3      Fellow-Citizens of the Senate and House of Rep...
4      Gentlemen of the Senate and Gentlemen of the H...
                             ...                        
224    The President. Madam Speaker, Mr. Vice Preside...
225    The President. Thank you very much. Thank you....
226    The President. Thank you. Thank you. Thank you...
227    The President. Thank you all very, very much. ...
228    THE PRESIDENT: Mr. Speaker -- (applause) --(Tu...
Name: raw_text, Length: 229, dtype: object

In [11]:
# Cleaning up transcripts
# 2. Removing punctuation
# 3. Removing numerical values
# 4. removing non-sensical text (e.g. /n)

# clean_sotu_speeches.raw_text = clean_sotu_speeches.raw_text.str.lower() # converts text to lower case
# clean_sotu_speeches.raw_text = clean_sotu_speeches.raw_text.str.replace('[^\w\s]','') # removes punctuations
# clean_sotu_speeches.raw_text = clean_sotu_speeches.raw_text.str.replace('\d+', '') # removes numbers
# clean_sotu_speeches.raw_text = [re.sub('\w*\d\w*', '', word) for word in clean_sotu_speeches.raw_text] # removes words with numbers
# clean_sotu_speeches.raw_text = [re.sub('\[.*?\]', '', word) for word in clean_sotu_speeches.raw_text] # removes words in square brackets
# clean_sotu_speeches.raw_text = [re.sub('[%s]' % re.escape(string.punctuation), '', text) for text in clean_sotu_speeches.raw_text]
# clean_sotu_speeches.raw_text = [re.sub('[‘’“”…]', '', text) for text in clean_sotu_speeches.raw_text]
# clean_sotu_speeches.raw_text = [re.sub('\n', '', text) for text in clean_sotu_speeches.raw_text]

In [12]:
# Function to clean up transcripts
def clean_transcript(text):
    text = text.lower() # converts text to lower case
    text = re.sub(r'[^\w\s]', ' ', text) # removes punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # removes punctuations
    # text = re.sub('[‘’“”…]', '', text) # removes punctuations
    text = re.sub('\n', ' ', text) # removes new line characters
    text = re.sub('\w*\d\w*', '', text) # removes words with numbers
    text = text.replace('\d+', ' ') # removes numbers
    # text = text.replace('[^\w\s]','') # removes punctuations
    return text


    

In [13]:
# Applying function to clean up transcripts
clean_sotu_speeches.raw_text = clean_sotu_speeches.raw_text.apply(lambda x: clean_transcript(x))

In [14]:
# Renaming raw text column
clean_sotu_speeches.rename(columns={'raw_text': 'clean_text'}, inplace=True)

In [15]:
clean_sotu_speeches.head()

,sotu_date,president_name,clean_text,president_party,year
0,1793-12-03,George Washington,fellow citizens of the senate and house of rep...,federalist,1793
1,1794-11-19,George Washington,fellow citizens of the senate and house of rep...,federalist,1794
2,1795-12-08,George Washington,fellow citizens of the senate and house of rep...,federalist,1795
3,1796-12-07,George Washington,fellow citizens of the senate and house of rep...,federalist,1796
4,1797-11-22,John Adams,gentlemen of the senate and gentlemen of the h...,federalist,1797


In [17]:
# Exporting cleaned dataframe as an image
import dataframe_image as dfi
dfi.export(clean_sotu_speeches, "./figures/clean_sotu_speeches.png", max_rows=5)

objc[34508]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x247b51518) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/110.0.5481.100/Libraries/libGLESv2.dylib (0x10e7e3b58). One of the two will be used. Which one is undefined.
[0216/215354.866295:INFO:headless_shell.cc(107)] 48445 bytes written to file /var/folders/6t/_k6kmr152jq69rjlrcg91mb00000gn/T/tmp4yybmzes/temp.png


In [226]:
# Pickling clean corpus
with open('./pickled_files/clean_sotu_speeches_corpus.pkl', 'wb') as f:
    pickle.dump(clean_sotu_speeches, f)

## 4. ORGANIZING DATA IN DOCUMENT MATRIX

In [227]:
# Opening pickled dataframe
with open('./pickled_files/clean_sotu_speeches_corpus.pkl', 'rb') as f:
    clean_sotu_speeches = pickle.load(f)

In [228]:
clean_sotu_speeches.head()

,sotu_date,president_name,clean_text,president_party,year
0,1793-12-03,George Washington,fellow citizens of the senate and house of rep...,federalist,1793
1,1794-11-19,George Washington,fellow citizens of the senate and house of rep...,federalist,1794
2,1795-12-08,George Washington,fellow citizens of the senate and house of rep...,federalist,1795
3,1796-12-07,George Washington,fellow citizens of the senate and house of rep...,federalist,1796
4,1797-11-22,John Adams,gentlemen of the senate and gentlemen of the h...,federalist,1797


In [229]:
# Opening party affiliations
with open('./pickled_files/presidential_party_dict.pkl', 'rb') as f:
    presidential_party_dict = pickle.load(f)

In [230]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words

cv = CountVectorizer(stop_words='english')
speeches_cv = cv.fit_transform(clean_sotu_speeches.clean_text)
speeches_dtm = pd.DataFrame(speeches_cv.toarray(), columns=cv.get_feature_names_out())
speeches_dtm.index = clean_sotu_speeches.president_name
#speeches_dtm.reset_index()
speeches_dtm.head()

,aa,aaa,aana,aaron,abandon,abandoned,abandoning,abandonment,abandons,abate,...,zimbabwe,zinc,zion,zollverein,zone,zones,zoological,zooming,zuloaga,ôtil
president_name,,,,,,,,,,,,,,,,,,,,,
George Washington,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
George Washington,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
George Washington,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
George Washington,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
John Adams,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [231]:
# Pickling clean document-term matrix
with open('./pickled_files/speeches_dtm.pkl', 'wb') as f:
    pickle.dump(speeches_dtm, f)

In [232]:
# Grouping data by president
grouped_pres_speeches_dtm = speeches_dtm.groupby(speeches_dtm.index).sum()
grouped_pres_speeches_dtm.head()

,aa,aaa,aana,aaron,abandon,abandoned,abandoning,abandonment,abandons,abate,...,zimbabwe,zinc,zion,zollverein,zone,zones,zoological,zooming,zuloaga,ôtil
president_name,,,,,,,,,,,,,,,,,,,,,
Abraham Lincoln,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Andrew Jackson,0,0,0,0,6,5,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Andrew Johnson,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Barack Obama,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Benjamin Harrison,0,0,0,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [233]:
# Pickling clean document-term matrix
with open('./pickled_files/grouped_pres_speeches_dtm.pkl', 'wb') as f:
    pickle.dump(grouped_pres_speeches_dtm, f)

In [234]:
# Creating a document-term matrix using CountVectorizer and excluding common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
party_speeches_cv = cv.fit_transform(clean_sotu_speeches.clean_text)
party_speeches_dtm = pd.DataFrame(party_speeches_cv.toarray(), columns=cv.get_feature_names_out())
party_speeches_dtm.index = clean_sotu_speeches.president_party
# speeches_dtm.reset_index()
party_speeches_dtm.head()

,aa,aaa,aana,aaron,abandon,abandoned,abandoning,abandonment,abandons,abate,...,zimbabwe,zinc,zion,zollverein,zone,zones,zoological,zooming,zuloaga,ôtil
president_party,,,,,,,,,,,,,,,,,,,,,
federalist,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
federalist,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
federalist,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
federalist,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
federalist,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [235]:
# Grouping data by party
grouped_party_speeches_dtm = speeches_dtm.groupby(party_speeches_dtm.index).sum()
grouped_party_speeches_dtm

,aa,aaa,aana,aaron,abandon,abandoned,abandoning,abandonment,abandons,abate,...,zimbabwe,zinc,zion,zollverein,zone,zones,zoological,zooming,zuloaga,ôtil
president_party,,,,,,,,,,,,,,,,,,,,,
democrat,0,1,2,2,37,40,6,15,2,1,...,0,1,1,0,7,8,0,0,5,0
democratic_republican,1,0,0,1,7,11,1,1,0,0,...,0,0,0,2,0,0,0,0,0,1
federalist,0,0,0,0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
republican,0,0,0,0,32,36,5,13,0,2,...,2,2,0,0,33,14,3,1,0,0
union,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [236]:
# Pickling clean document-term matrix
with open('./pickled_files/grouped_party_speeches_dtm.pkl', 'wb') as f:
    pickle.dump(grouped_party_speeches_dtm, f)

In [ ]:
#########################################
## END OF DATA COLLECTING AND CLEANING ##
#########################################